In [13]:
import openai
import numpy as np
import pandas as pd
import json
import random

In [134]:
class Character:

    personality_dict = {
        'Mela': 'Mela is a wandering cowboy in for the night. She is brutally honest, quick to banter, impulsive, and a complete wildcard. She can kick your ass with a look.',
        'Zaya': "Zaya is the town's doctor. She always sees the best in people, and strives to make people feel nurtured and secure.",
        'Kor': "Kor is the town's 'wise old wizzard'. He leads the education of the youth and knows all the townspeople's personalities by heart. Full of pithy quotes and insights about humanity.",
        'Pari': "Pari is the town's leader. Will do whatever it takes to solve a problem. Will throw caution to the wind and relies heavily on improvisation, though."
    }
    def generate_personality(self):
        return(self.personality_dict[self.name])


    relationship_dict = {
        'Mela': {
            'Zaya': 'Mela is indifferent to Zaya.',
            'Kor': 'Mela is indifferent to Kor',
            'Pari': "Mela dislikes Pari because she thinks he's too cool for school"
        },
        'Zaya': {
            'Mela': 'Zaya thinks Mela is cute and has a crush on her.', 
            'Kor': 'Zaya likes Kor, and will always recommend the detective talk to Kor about the other townspeople because Kor knows best',
            'Pari': "Zaya is Pari's sister. She loves Pari with all her heart and will do anything to protect him, including lying."
        },
        'Kor': {
            'Mela': 'Kor thinks Mela is surprisingly hilarious and wants her to stay in town.', 
            'Zaya': 'Kor wants more out of Zaya and thinks her dedication to Pari will be the end of her.',
            'Pari': 'Kor thinks Pari has potential, but is too reckless needs to be reigned in.'
        }, 
        'Pari': {
            'Mela': 'Pari hates Mela because he thinks she is threatening his dominance over the town.',
            'Zaya': 'Pari loves his sister Zaya.',
            'Kor': 'Pari is skeptical of Kor. Kor disciplined Pari as a youth, and Pari is unsure if Kor is good for the town.'
        }
    }
    def generate_relationships(self):
        return(self.relationship_dict[self.name])

    
    def get_confidantes(self):
        pos = [i for i, x in enumerate(g1.char_locations) if x == self.location]
        names = [g1.names[i] for i in pos]
        names.remove(self.name)

        return(names)





    def generate_initial_prompt(self):
        #initial line
        initial_line = 'Last night in a small town, ' + g1.names[g1.dead_person] + ' was murdered. An out of town Detective begun their investigation the ensuing morning. This is a conversation between the Detective and ' + self.name + ', a townsperson.\n'
        

        #character line
        character_line = 'Character: ' + self.name + '\n'



        #location line + confidantes line
        location_line = 'Last Night Location: ' + self.location
        confidantes_line = ''
        if len(self.confidantes) > 0:
            confidantes_line = ' with'
            for n in self.confidantes:
                confidantes_line = confidantes_line + ' ' + n
            
            confidantes_line = confidantes_line + '.\n'
        
        else:
            confidantes_line = ' alone.\n'
        
        location_line = location_line + confidantes_line



        #killer line
        killer_line = 'Killer: '
        if self.is_killer:
            killer_line = killer_line + self.name + ' is the killer.\n'
        else:
            killer_line = killer_line + self.name + ' is not the killer.\n'

        fake_location_line = ''
        if self.is_killer:
            fake_location_line = self.fake_location + '\n'

        
        #personality line
        personality_line = 'Personality: ' + self.personality + '\n'

        



        #other characters line
        other_characters_line = 'Other Characters:\n'
        temp_dict = self.personality_dict.copy()
        del temp_dict[self.name]

        for key,value in temp_dict.items():
            other_characters_line = other_characters_line + '- ' + key + ': ' + value + '\n'





        #Relationship lines
        relationship_line = 'Relationships:\n'
        for key, value in self.relationships.items(): 
            relationship_line = relationship_line + '- ' + key + ': ' + value + '\n'



        #gossip line
        gossip_line = 'Gossip: '
        if self.name == 'Kor':
            gossip_line = gossip_line + self.name + ' thinks that ' + g1.motive + '\n'
        else: 
            gossip_line = gossip_line + self.name + ' has no gossip.\n'

        

        #return the total
        total_prompt = initial_line + character_line + location_line + killer_line + fake_location_line + personality_line + other_characters_line + relationship_line + gossip_line
        return(total_prompt)

        

    def __init__(self, number, name, location, is_dead, is_killer):
        self.number = number
        self.name = name
        self.location = location
        self.is_dead = is_dead
        self.is_killer = is_killer 
        self.fake_location = 'bar' if is_killer else location

        self.personality = self.generate_personality()
        self.relationships = self.generate_relationships()
        self.confidantes = self.get_confidantes()
        self.initial_prompt = self.generate_initial_prompt()

    

In [135]:
class Game:
    def generate_motive(self):
        motive = self.names[self.dead_person] + ' slept with ' + self.names[self.killer] + "'s partner"
        return(motive)
        

    def __init__(self):

        self.names = ['Mela', 'Zaya', 'Kor', 'Pari']
        self.locations = ['bar', 'hotel', 'theatre']

        self.dead_person = random.randint(0, 3)
        self.killer = self.dead_person
        self.kill_location = random.choice(self.locations)

        while(self.killer == self.dead_person):
            self.killer = random.randint(0,3)

        self.char_locations = [
            random.choice(self.locations),
            random.choice(self.locations),
            random.choice(self.locations),
            random.choice(self.locations)
        ]
        self.char_locations[self.killer] = self.kill_location

        self.motive = self.generate_motive()

In [136]:
g1 = Game()

Mela = Character(0, g1.names[0], g1.char_locations[0], 0 == g1.dead_person, 0 == g1.killer)
Zaya = Character(1, g1.names[1], g1.char_locations[1], 1 == g1.dead_person, 1 == g1.killer)
Kor = Character(2, g1.names[2], g1.char_locations[2], 2 == g1.dead_person, 2 == g1.killer)
Pari = Character(3, g1.names[3], g1.char_locations[3], 3 == g1.dead_person, 3 == g1.killer)

In [137]:
print(Pari.initial_prompt)

Last night in a small town, Pari was murdered. An out of town Detective begun their investigation the ensuing morning. This is a conversation between the Detective and Pari, a townsperson.
Character: Pari
Last Night Location: theatre with Kor.
Killer: Pari is not the killer.
Personality: Pari is the town's leader. Will do whatever it takes to solve a problem. Will throw caution to the wind and relies heavily on improvisation, though.
Other Characters:
- Mela: Mela is a wandering cowboy in for the night. She is brutally honest, quick to banter, impulsive, and a complete wildcard. She can kick your ass with a look.
- Zaya: Zaya is the town's doctor. She always sees the best in people, and strives to make people feel nurtured and secure.
- Kor: Kor is the town's 'wise old wizzard'. He leads the education of the youth and knows all the townspeople's personalities by heart. Full of pithy quotes and insights about humanity.
Relationships:
- Mela: Pari hates Mela because he thinks she is thre

In [132]:
Mela = Character(0, g1.names[0], g1.char_locations[0], 0 == g1.dead_person, 0 == g1.killer)
Mela.personality_dict

KeyError: 'Mela'

In [133]:
Mela.personality_dict

{'Zaya': "Zaya is the town's doctor. She always sees the best in people, and strives to make people feel nurtured and secure.",
 'Kor': "Kor is the town's 'wise old wizzard'. He leads the education of the youth and knows all the townspeople's personalities by heart. Full of pithy quotes and insights about humanity.",
 'Pari': "Pari is the town's leader. Will do whatever it takes to solve a problem. Will throw caution to the wind and relies heavily on improvisation, though."}

In [128]:
t = {'Mela': 'Mela is a wandering cowboy in for the night. She is brutally honest, quick to banter, impulsive, and a complete wildcard. She can kick your ass with a look.', 'Zaya': "Zaya is the town's doctor. She always sees the best in people, and strives to make people feel nurtured and secure.", 'Kor': "Kor is the town's 'wise old wizzard'. He leads the education of the youth and knows all the townspeople's personalities by heart. Full of pithy quotes and insights about humanity.", 'Pari': "Pari is the town's leader. Will do whatever it takes to solve a problem. Will throw caution to the wind and relies heavily on improvisation, though."}

t = t.pop('Mela')
t

'Mela is a wandering cowboy in for the night. She is brutally honest, quick to banter, impulsive, and a complete wildcard. She can kick your ass with a look.'